In [13]:
import json
import pandas as pd
import os
import requests
from tqdm import tqdm
file_path = "sky_conditions_20240701_20250830.json"

In [14]:
with open(file_path, 'r') as f:
    data = json.load(f)

In [15]:
observations = [feat['properties'] for feat in data['features']]
# print(json.dumps(observations[0], indent=2))
# print('skyconditionsSouthPhotoUrl' in observations[0])

In [16]:
cloud_types = [
    "skyconditionsAltocumulus",
    "skyconditionsAltostratus",
    "skyconditionsCirrocumulus",
    "skyconditionsCirrostratus",
    "skyconditionsCirrus",
    "skyconditionsCumulonimbus",
    "skyconditionsCumulus",
    "skyconditionsNimbostratus",
    "skyconditionsStratocumulus",
    "skyconditionsStratus"
]
def has_cloud_type(obs):
    return any(obs[cloud_type] == 'true' for cloud_type in cloud_types)

photo_types = [
    "skyconditionsNorthPhotoUrl",
    "skyconditionsSouthPhotoUrl",
    "skyconditionsEastPhotoUrl",
    "skyconditionsWestPhotoUrl",
    "skyconditionsUpwardPhotoUrl"
]
def has_photos(obs):
    return any(obs[photo_type] != 'null' for photo_type in photo_types)
def count_photos(obs):
    return sum(int(obs[photo_type] != 'null') for photo_type in photo_types)

print(sum(has_cloud_type(obs) and has_photos(obs) for obs in observations))
print(sum(int(has_cloud_type(obs))*count_photos(obs) for obs in observations))

77050
341695


In [17]:
rows = []
for obs in observations:
    if not (has_cloud_type(obs) and has_photos(obs)):
        continue

    one_hot = {ct[13:].lower(): int(obs[ct] == 'true') for ct in cloud_types}
    for pt in photo_types:
        photo_url = obs.get(pt)
        # if photo_url == 'pending approval':
        #     continue
        # if photo_url[-3:] in {'peg', 'png', 'ted', 'JPG'}:
        #     continue
        if photo_url[-3:] != 'jpg':
            continue
        if photo_url and photo_url != 'null':
            row = {'photo_url': photo_url}
            row.update(one_hot)
            rows.append(row)

print(f'number of observations: {len(rows)}')
s = set()
for row in rows:
    s.add(row['photo_url'][-3:])
print(f'types of data: {s}')

number of observations: 334540
types of data: {'jpg'}


In [18]:
df = pd.DataFrame(rows)
print(df.head())

                                           photo_url  altocumulus  \
0  https://data.globe.gov/system/photos/2025/08/3...            1   
1  https://data.globe.gov/system/photos/2025/08/3...            1   
2  https://data.globe.gov/system/photos/2025/08/3...            0   
3  https://data.globe.gov/system/photos/2025/08/3...            0   
4  https://data.globe.gov/system/photos/2025/08/3...            0   

   altostratus  cirrocumulus  cirrostratus  cirrus  cumulonimbus  cumulus  \
0            1             0             0       0             0        1   
1            1             0             0       0             0        1   
2            1             0             0       0             0        0   
3            1             0             0       0             0        0   
4            1             0             0       0             0        0   

   nimbostratus  stratocumulus  stratus  
0             0              0        1  
1             0              0        

In [28]:
subset = df.head(100).copy()
# subset.to_csv('clouds_subset.csv', index=False)
os.makedirs('cloud_images', exist_ok=True)

image_paths = []
for i, url in tqdm(enumerate(subset['photo_url']), total=len(subset)):
    try: 
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            filename = f'cloud_{i}.jpg'
            filepath = os.path.join('cloud_images', filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            image_paths.append(filepath)
        else:
            image_paths.append(None)
    except Exception as e:
        image_paths.append(None)
subset['local_path'] = image_paths
# subset.to_csv('clouds_subset.csv', index=False)

100%|█████████████████████████████████████████| 100/100 [01:06<00:00,  1.51it/s]


In [25]:
subset.head()

,photo_url,altocumulus,altostratus,cirrocumulus,cirrostratus,cirrus,cumulonimbus,cumulus,nimbostratus,stratocumulus,stratus,local_path
0,https://data.globe.gov/system/photos/2025/08/3...,1,1,0,0,0,0,1,0,0,1,cloud_images/cloud_0.jpg
1,https://data.globe.gov/system/photos/2025/08/3...,1,1,0,0,0,0,1,0,0,1,cloud_images/cloud_1.jpg
2,https://data.globe.gov/system/photos/2025/08/3...,0,1,0,0,0,0,0,0,0,0,cloud_images/cloud_2.jpg
3,https://data.globe.gov/system/photos/2025/08/3...,0,1,0,0,0,0,0,0,0,0,cloud_images/cloud_3.jpg
4,https://data.globe.gov/system/photos/2025/08/3...,0,1,0,0,0,0,0,0,0,0,cloud_images/cloud_4.jpg


In [30]:
val = df.iloc[2000:3000].copy()
val.to_csv('clouds_val.csv', index=False)
os.makedirs('cloud_images', exist_ok=True)

image_paths = []
for i, url in tqdm(enumerate(val['photo_url']), total=len(val)):
    try: 
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            filename = f'cloud_{2000+i}.jpg'
            filepath = os.path.join('cloud_images', filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            image_paths.append(filepath)
        else:
            image_paths.append(None)
    except Exception as e:
        image_paths.append(None)
val['local_path'] = image_paths
val.to_csv('clouds_val.csv', index=False)

100%|███████████████████████████████████████| 1000/1000 [10:07<00:00,  1.65it/s]
